# <div align="center" style="color: #ff5733;">Recovery Rate As per the Bucket</div>

In [1]:
# %% [markdown]
# # Jupyter Notebook Loading Header
#
# This is a custom loading header for Jupyter Notebooks in Visual Studio Code.
# It includes common imports and settings to get you start'{ed}' quickly.

# %% [markdown]
## Import Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from google.cloud import bigquery
import os
path = r'C:\Users\DwaipayanChakroborti\AppData\Roaming\gcloud\legacy_cr'{ed}'entials\dchakroborti@tonikbank.com\adc.json'
os.environ['GOOGLE_APPLICATION_CR'{ed}'ENTIALS'] = path
client = bigquery.Client(project='prj-prod-dataplatform')

# %% [markdown]
## Configure Settings
# Set options or configurations as ne'{ed}''{ed}'
# Example: pd.set_option('display.max_columns', None)

In [2]:
# Setup the Month Start and End Date

sd = '2024-03-01'
ed = '2024-03-17'

In [4]:
sq = f"""
with Overdue as 
(select bucketDate
, DATE'{sd}' wkstartdate
,  DATE'{ed}' wkenddate
, loanStatus
, loanAccountNumber
,case  
  when coalesce(Max_current_DPD,0) <= 0 then 'B. Not Delinquent'
  when coalesce(Max_current_DPD,0) >= 1 and coalesce(Max_current_DPD, 0) <= 30 then 'C. 1-30DPD' 
  when coalesce(Max_current_DPD, 0) >=31 and coalesce(Max_current_DPD, 0) <= 60 then 'D. 31-60DPD'
  when coalesce(Max_current_DPD, 0) >=61 and coalesce(Max_current_DPD, 0) <= 90 then 'E. 61-90DPD'
  when coalesce(Max_current_DPD, 0) >=91 and coalesce(Max_current_DPD, 0) <= 120 then 'F. 91-120DPD'
  when coalesce(Max_current_DPD, 0) >=121 and coalesce(Max_current_DPD, 0) <= 150 then 'G. 121-150DPD'
  when coalesce(Max_current_DPD, 0) >=151 and coalesce(Max_current_DPD, 0) <= 180 then 'H. 151-180DPD'
  when coalesce(Max_current_DPD, 0) >=181 and coalesce(Max_current_DPD, 0) <= 360 then 'I. 181-360DPD'
  when coalesce(Max_current_DPD, 0) >=361 then 'J. >360DPD' end  bucketbin
,case
  when coalesce(Max_current_DPD,0) <= 0 then 0
  when coalesce(Max_current_DPD,0) >= 1 and coalesce(Max_current_DPD, 0) <= 30 then 1 
  when coalesce(Max_current_DPD, 0) >=31 and coalesce(Max_current_DPD, 0) <= 60 then 2
  when coalesce(Max_current_DPD, 0) >=61 and coalesce(Max_current_DPD, 0) <= 90 then 3
  when coalesce(Max_current_DPD, 0) >=91 and coalesce(Max_current_DPD, 0) <= 120 then 4
  when coalesce(Max_current_DPD, 0) >=121 and coalesce(Max_current_DPD, 0) <= 150 then 5
  when coalesce(Max_current_DPD, 0) >=151 and coalesce(Max_current_DPD, 0) <= 180 then 6
  when coalesce(Max_current_DPD, 0) >=181 and coalesce(Max_current_DPD, 0) <= 360 then 7
  when coalesce(Max_current_DPD, 0) >=361 then 8 end  bucket,
sum(Total_Outstanding_Amount) totaloutstandingamount,
sum(Total_Overdue_Amount) Total_Overdue_Amount
from `risk_credit_mis.loan_bucket_flow_report_core`
where bucketDate = DATE'{sd}'
and loanStatus in ('Normal', 'In Arrears')
and Total_Outstanding_Amount is not null
and Total_Outstanding_Amount > 0
group by 1, 2,3,4,5,6,7
)
,
overduebase as
(select bucketDate, wkstartdate,wkenddate, loanAccountNumber, bucket
, sum(totaloutstandingamount)totaloutstandingamount 
, sum(case when loanStatus = 'In Arrears' then totaloutstandingamount else 0 end) inArearstotaloutstandingamount
, sum(case when loanStatus = 'Normal' then totaloutstandingamount else 0 end) Normaltotaloutstandingamount
, sum(case when loanStatus = 'In Arrears' then Total_Overdue_Amount else 0 end) inArearstoTotal_Overdue_Amount
, sum(case when loanStatus = 'Normal' then Total_Overdue_Amount else 0 end) NormalTotal_Overdue_Amount
from Overdue
group by 1,2,3,4,5
)
,
payments as 
(select  DATE'{sd}' wkstartdate, DATE'{ed}' wkenddate,  loanAccountNumber, lastPaymentDate,isDelinquent, isCurrentDelinquent,
sum(installmentPaidAmount) payment
from `risk_credit_mis.loan_installments_table`
where coalesce(lastPaymentDate, date('3000-01-01')) between  DATE'{sd}' and DATE'{ed}'
and isDelinquent = 1
and coalesce(installmentPaidAmount, 0) > 0
group by 1,2,3,4,5,6
), 
paymentbase as  
(select wkstartdate, wkenddate, loanAccountNumber
, count(distinct lastPaymentDate) noofdayspaymentreceived 
, sum(case when isDelinquent = 1 and isCurrentDelinquent = 0 then 1 else 0 end) Normalized_cnt
, sum(case when isDelinquent = 1 and isCurrentDelinquent = 1 then 1 else 0 end) paymentreceivedbutstilldelinquent_cnt
, sum(case when isDelinquent = 1 and isCurrentDelinquent = 0 then payment else 0 end) Normalized
, sum(case when isDelinquent = 1 and isCurrentDelinquent = 1 then payment else 0 end) paymentreceivedbutstilldelinquent
, sum(payment) payment
from payments 
group by 1,2,3
)
select 
overduebase.bucketDate
, overduebase.wkstartdate
, overduebase.wkenddate
, bucket
, count(distinct overduebase.loanAccountNumber ) cntloanoverdue
, sum(overduebase.totaloutstandingamount) totaloutstandingamount
, sum(overduebase.inArearstotaloutstandingamount) inArearstotaloutstandingamount
, sum(overduebase.Normaltotaloutstandingamount) Normaltotaloutstandingamount
, sum(overduebase.inArearstoTotal_Overdue_Amount) inArearstoTotal_Overdue_Amount
, sum(overduebase.NormalTotal_Overdue_Amount)NormalTotal_Overdue_Amount
, count(distinct paymentbase.loanAccountNumber) cnt_loanpaymentsreceived
, max(noofdayspaymentreceived) noofdayspaymentreceived
, sum(Normalized_cnt) Normalized_cnt
, sum(Normalized)Normalized
, sum(paymentreceivedbutstilldelinquent_cnt)paymentreceivedbutstilldelinquent_cnt
, sum(paymentreceivedbutstilldelinquent) paymentreceivedbutstilldelinquent
, sum(payment)payment
from overduebase
left join paymentbase on overduebase.loanAccountNumber = paymentbase.loanAccountNumber 
group by 1,2,3,4

"""
dfrecovery = client.query(sq).to_dataframe(progress_bar_type='tqdm')
dfrecovery['paymentsfromoverdue'] = dfrecovery['Normalized'] + dfrecovery['paymentreceivedbutstilldelinquent']
dfrecovery['recoveryrate'] = (dfrecovery['Normalized'] + dfrecovery['paymentreceivedbutstilldelinquent'])/dfrecovery['inArearstoTotal_Overdue_Amount']
dfrecovery

Job ID 55bbf50a-0233-4304-8a64-85c43c4a5b13 successfully executed: 100%|██████████|
Downloading: 100%|██████████|


,bucketDate,wkstartdate,wkenddate,bucket,cntloanoverdue,totaloutstandingamount,inArearstotaloutstandingamount,Normaltotaloutstandingamount,inArearstoTotal_Overdue_Amount,NormalTotal_Overdue_Amount,cnt_loanpaymentsreceived,noofdayspaymentreceived,Normalized_cnt,Normalized,paymentreceivedbutstilldelinquent_cnt,paymentreceivedbutstilldelinquent,payment,paymentsfromoverdue,recoveryrate
0,2024-03-01,2024-03-01,2024-03-17,8,8716,2.674614e+08,2.674614e+08,0.000000e+00,2.378988e+08,0.00,0,<NA>,<NA>,NaN,<NA>,NaN,NaN,NaN,NaN
1,2024-03-01,2024-03-01,2024-03-17,0,39502,5.828238e+08,0.000000e+00,5.828238e+08,0.000000e+00,228439.84,3030,2,1858,4422061.27,1182,221355.61,4643416.88,4643416.88,inf
2,2024-03-01,2024-03-01,2024-03-17,2,1733,3.330298e+07,3.330298e+07,0.000000e+00,9.182980e+06,0.00,186,2,165,692148.73,120,151392.76,843541.49,843541.49,0.091859
3,2024-03-01,2024-03-01,2024-03-17,7,6668,1.943748e+08,1.943748e+08,0.000000e+00,1.345131e+08,0.00,28,1,19,103410.88,26,53542.11,156952.99,156952.99,0.001167
4,2024-03-01,2024-03-01,2024-03-17,4,1480,3.555020e+07,3.555020e+07,0.000000e+00,1.618280e+07,0.00,35,2,29,124071.66,28,34115.45,158187.11,158187.11,0.009775
5,2024-03-01,2024-03-01,2024-03-17,6,1574,4.563584e+07,4.563584e+07,0.000000e+00,2.607420e+07,0.00,37,2,24,177590.68,31,39836.25,217426.93,217426.93,0.008339
6,2024-03-01,2024-03-01,2024-03-17,1,3730,6.393349e+07,6.393349e+07,0.000000e+00,9.018021e+06,0.00,1979,2,1541,3722614.84,594,228375.66,3950990.50,3950990.50,0.438122
7,2024-03-01,2024-03-01,2024-03-17,5,1601,4.507110e+07,4.507110e+07,0.000000e+00,2.347318e+07,0.00,61,2,46,311029.19,49,86043.22,397072.41,397072.41,0.016916
8,2024-03-01,2024-03-01,2024-03-17,3,1324,3.231576e+07,3.231576e+07,0.000000e+00,1.115443e+07,0.00,50,1,38,240179.64,39,61542.74,301722.38,301722.38,0.027050


In [5]:
dfrecovery[['bucketDate','wkstartdate','wkenddate','bucket', 'recoveryrate']].to_excel("Bucketbasedrecoveryrate_march2024.xlsx", index = False, sheet_name = 'Data')